In [1]:
import selenium
from selenium import webdriver


from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time

In [2]:
#mangoplateURL="https://www.mangoplate.com/search/서울?keyword=서울&page=1"

#동적 사이트 크롤링이라서 requests 대신 Selenium 사용함 -> 직접 서버와 통신 가능 


#path= r"C:\Users\cxz01\OneDrive\바탕 화면\soongsil university\데이터사이언스\팀프로젝트\DS_Univ_5\web_crawling\chromedriver_win32\chromedriver.exe"
#browser = webdriver.Chrome(path)
#browser.get(mangoplateURL)

In [15]:
mangoData=pd.DataFrame(columns=["title","location","estimation","viewCount","reviewCount"])
path= r"C:\Users\cxz01\OneDrive\바탕 화면\soongsil university\데이터사이언스\팀프로젝트\DS_Univ_5\web_crawling\chromedriver_win32\chromedriver.exe"
browser = webdriver.Chrome(path)


#검색 총 page 개수: 50
for page in range(1,51):
    mangoplateURL=f"https://www.mangoplate.com/search/서울?keyword=서울&page={page}"
    browser.get(mangoplateURL)
    time.sleep(2)
    pageSource = BeautifulSoup(browser.page_source)
    infoList=pageSource.find_all("div", attrs = {"class":"info"})
    for info in infoList[0:20]:
        title = info.find("h2", attrs = {"class" : "title"}).get_text()
        detailURL = info.find("a")["href"]
        try:
            estimation = info.find("strong", attrs = {"class" : "point search_point"}).get_text()
        except:
            estimation = info.find("strong", attrs = {"class" : "point search_point expected"}).get_text()
        viewCount = info.find("span", attrs = {"class" : "view_count"}).get_text()
        reviewCount = info.find("span", attrs = {"class" : "review_count"}).get_text()
        
        title = title.strip()
        detailURL = "https://www.mangoplate.com" + detailURL
        estimation = float(estimation)
        viewCount = float("".join(viewCount.split(",")))
        reviewCount = float("".join(reviewCount.split(",")))
        
        browser.get(detailURL)
        time.sleep(1)
        pageSource = BeautifulSoup(browser.page_source)
        location = pageSource.find("span", attrs = {"class" : "Restaurant__InfoAddress--Text"}).get_text()
        
        newData = {"title" : [title], "location" : [location], "estimation" : [estimation], "viewCount": [viewCount], "reviewCount" : [reviewCount]}
        newData = pd.DataFrame(newData)
        mangoData = pd.concat((mangoData, newData)).reset_index(drop = True)


C:\Users\cxz01\AppData\Local\Temp\ipykernel_11404\4074527112.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(path)


In [17]:
mangoData

,title,location,estimation,viewCount,reviewCount
0,스시카나에,서울시 강남구 청담동 96-22 노노빌딩 2F,4.8,402012.0,110.0
1,미라이,서울시 강남구 신사동 557-25 끌레르빌 1F,4.8,524763.0,131.0
2,시라카와,서울시 강남구 신사동 664-24 1F,4.8,332121.0,75.0
3,세이류,서울시 송파구 신천동 11-4 B1 B35호,4.8,163681.0,36.0
4,이안정,서울시 마포구 상수동 315-3 2F 101호,4.8,13697.0,21.0
...,...,...,...,...,...
995,파인앤코,서울시 강남구 신사동 657 B1,4.3,13589.0,15.0
996,물고기주택,서울시 용산구 한강로2가 87-8,4.3,43565.0,24.0
997,네기우나기야,서울시 송파구 신천동 29-1 소피텔앰배서더 2F,4.3,28318.0,14.0
998,스시야스다,서울시 서초구 서초동 1675-10 지에스타워 203호,4.3,11492.0,6.0


In [2]:
mangoData.to_csv('mangoData.csv', index=False, encoding='utf-8')

NameError: name 'mangoData' is not defined

In [5]:
geoData = pd.read_excel(r"C:\Users\cxz01\OneDrive\바탕 화면\soongsil university\데이터사이언스\팀프로젝트\DS_Univ_5\web_crawling\mango\geoData.xlsx")
geoData = geoData[geoData["시도"] == "서울특별시"]
geoData = geoData.drop_duplicates(subset = ["시군구", "읍면동"])
geoData["geo"] = geoData["시도"] + " " + geoData["시군구"] + " " + geoData["읍면동"]
geoData


,코드,시도,시군구,읍면동,하위,위도,경도,코드종류,geo
0,1111051500,서울특별시,종로구,청운효자동,NaN,37.584009,126.970626,H,서울특별시 종로구 청운효자동
1,1111053000,서울특별시,종로구,사직동,NaN,37.575408,126.965944,H,서울특별시 종로구 사직동
2,1111054000,서울특별시,종로구,삼청동,NaN,37.590758,126.980996,H,서울특별시 종로구 삼청동
3,1111055000,서울특별시,종로구,부암동,NaN,37.594768,126.965574,H,서울특별시 종로구 부암동
4,1111056000,서울특별시,종로구,평창동,NaN,37.613029,126.974485,H,서울특별시 종로구 평창동
...,...,...,...,...,...,...,...,...,...
4005,1174010200,서울특별시,강동구,고덕동,NaN,37.561068,127.160582,B,서울특별시 강동구 고덕동
4008,1174010600,서울특별시,강동구,둔촌동,NaN,37.531157,127.142891,B,서울특별시 강동구 둔촌동
4009,1174010700,서울특별시,강동구,암사동,NaN,37.552497,127.133010,B,서울특별시 강동구 암사동
4010,1174010800,서울특별시,강동구,성내동,NaN,37.531338,127.129174,B,서울특별시 강동구 성내동


In [8]:
mapData = pd.read_csv(r"C:\Users\cxz01\OneDrive\바탕 화면\soongsil university\데이터사이언스\팀프로젝트\DS_Univ_5\web_crawling\mango\mangoData.csv")
mapData["geo_1"] = "서울특별시"
mapData["geo_2"] = mapData["location"].str.split(" ").str[1]
mapData["geo_3"] = mapData["location"].str.split(" ").str[2]
mapData["geo"] = mapData["geo_1"] + " " + mapData["geo_2"] + " " + mapData["geo_3"]

mapData = pd.merge(left = mapData, right = geoData[["위도", "경도", "geo"]], how = "left", left_on = "geo", right_on = "geo")
mapData = mapData.dropna(subset = ["위도","경도"]).reset_index(drop=True)
mapData

,title,location,estimation,viewCount,reviewCount,geo_1,geo_2,geo_3,geo,위도,경도
0,스시카나에,서울시 강남구 청담동 96-22 노노빌딩 2F,4.8,402012.0,110.0,서울특별시,강남구,청담동,서울특별시 강남구 청담동,37.524399,127.050457
1,미라이,서울시 강남구 신사동 557-25 끌레르빌 1F,4.8,524763.0,131.0,서울특별시,강남구,신사동,서울특별시 강남구 신사동,37.523807,127.026492
2,시라카와,서울시 강남구 신사동 664-24 1F,4.8,332121.0,75.0,서울특별시,강남구,신사동,서울특별시 강남구 신사동,37.523807,127.026492
3,세이류,서울시 송파구 신천동 11-4 B1 B35호,4.8,163681.0,36.0,서울특별시,송파구,신천동,서울특별시 송파구 신천동,37.517302,127.104967
4,이안정,서울시 마포구 상수동 315-3 2F 101호,4.8,13697.0,21.0,서울특별시,마포구,상수동,서울특별시 마포구 상수동,37.547431,126.922946
...,...,...,...,...,...,...,...,...,...,...,...
977,파인앤코,서울시 강남구 신사동 657 B1,4.3,13589.0,15.0,서울특별시,강남구,신사동,서울특별시 강남구 신사동,37.523807,127.026492
978,물고기주택,서울시 용산구 한강로2가 87-8,4.3,43565.0,24.0,서울특별시,용산구,한강로2가,서울특별시 용산구 한강로2가,37.531715,126.970242
979,네기우나기야,서울시 송파구 신천동 29-1 소피텔앰배서더 2F,4.3,28318.0,14.0,서울특별시,송파구,신천동,서울특별시 송파구 신천동,37.517302,127.104967
980,스시야스다,서울시 서초구 서초동 1675-10 지에스타워 203호,4.3,11492.0,6.0,서울특별시,서초구,서초동,서울특별시 서초구 서초동,37.488514,127.016222


In [9]:
mapData.to_csv('mangoMapData.csv', index=False, encoding='utf-8')

In [10]:
mapData = mapData[['estimation','reviewCount','위도','경도']]

In [11]:
mapData

,estimation,reviewCount,위도,경도
0,4.8,110.0,37.524399,127.050457
1,4.8,131.0,37.523807,127.026492
2,4.8,75.0,37.523807,127.026492
3,4.8,36.0,37.517302,127.104967
4,4.8,21.0,37.547431,126.922946
...,...,...,...,...
977,4.3,15.0,37.523807,127.026492
978,4.3,24.0,37.531715,126.970242
979,4.3,14.0,37.517302,127.104967
980,4.3,6.0,37.488514,127.016222


In [12]:
mapData.to_csv('mangoMapData2.csv', index=False, encoding='utf-8')